# Przygotowanie danych

### Wczytanie niezbędnych bibliotek

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from dateutil import parser
import dateparser

import re
import requests
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer


### Wczytanie danych

In [2]:
df = pd.read_csv('df_nowe.csv')
df.shape[0]

633

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Nazwa oferty,Lokalizacja,Data dodania,Powierzchnia w m²,Cena za 1m²,Kwota,Link,Poziom,Umeblowane,Liczba pokoi,Opis
0,0,0,Mieszkanie do sprzedaży Katowice Bogucice 2 pok.,"Katowice, Bogucice",Dzisiaj o 14:16,"42,70 m²",8173.3 zł/m²,349 000 zł,https://www.otodom.pl/pl/oferta/mieszkanie-do-...,1/10,do zamieszkania,2,Oferujemy do sprzedaży 2 pokojowe mieszkanie o...
1,1,1,Okazja! Super mieszkanie ul. Adama! 62 m2 5 pi...,"Katowice, Giszowiec",Dzisiaj o 17:24,62 m²,5951.61 zł/m²,369 000 zł,https://www.olx.pl/d/oferta/okazja-super-miesz...,5,Nie,3 pokoje,"Agencje proszę nie dzwonić, chyba ze z zainter..."
2,2,2,SUPERJEDNOSTKA - zachodnie mieszkanie do wejścia,Katowice,12 grudnia 2023,"47,40 m²",9978.9 zł/m²,473 000 zł,https://www.otodom.pl/pl/oferta/superjednostka...,7/15,do zamieszkania,2,Mamy przyjemność przedstawić Państwu mieszkani...
3,3,3,Przestronne 3 pok.mieszkanie przy Parku Kościu...,Katowice,07 grudnia 2023,"73,38 m²",5444.26 zł/m²,399 500 zł,https://www.otodom.pl/pl/oferta/przestronne-3-...,2/4,do remontu,3,MOŻLIWOŚĆ ZAKUPU MIESZKANIA W RAMACH RZĄDOWEGO...
4,5,5,"Znakomite 3 pok. mieszkanie 53,4m2 Katowice-Mu...","Katowice, Murcki",25 listopada 2023,"53,37 m²",8993.82 zł/m²,480 000 zł,https://www.otodom.pl/pl/oferta/znakomite-3-po...,2/3,do zamieszkania,3,"Znakomite 3 pok. mieszkanie 53,4m2 Katowice-Mu..."


### Sprawdzenie, czy faktycznie wszystkie oferty pochodzą z miasta Katowice


In [7]:
df[~df['Lokalizacja'].str.contains('Katowice')].head()

,Unnamed: 0.1,Unnamed: 0,Nazwa oferty,Lokalizacja,Data dodania,Powierzchnia w m²,Cena za 1m²,Kwota,Link,Poziom,Umeblowane,Liczba pokoi,Opis
603,776,776,Przestronne mieszkanie z widokiem! Ogromny salon!,Dąbrowa Górnicza,21 listopada 2023,"76,50 m²",6784.31 zł/m²,519 000 zł,https://www.otodom.pl/pl/oferta/przestronne-mi...,parter/2,do zamieszkania,3,Oferuję na sprzedaż mieszkanie w Śródmieściu B...
604,777,777,Mieszkanie w prestiżowej dzielnicy Bytomia na ...,Bytom,09 grudnia 2023,41 m²,5487.8 zł/m²,225 000 złdo negocjacji,https://www.olx.pl/d/oferta/mieszkanie-w-prest...,2/3,do zamieszkania,2,"Przestronne, dwupokojowe mieszkanie w Bytomiu...."
605,778,778,Komfortowe mieszkanie Bytom Śródmieście,Bytom,Dzisiaj o 06:14,"68,84 m²",5360.26 zł/m²,369 000 zł,https://www.otodom.pl/pl/oferta/komfortowe-mie...,parter/3,do remontu,3,[b]Posiadamy w ofercie na sprzedaż mieszkanie ...
606,780,780,Mieszkanie 3pok. Bytom-Wożniaka,Bytom,07 grudnia 2023,63 m²,3634.92 zł/m²,229 000 zł,https://www.otodom.pl/pl/oferta/mieszkanie-3po...,3/3,do zamieszkania,3,Przedstawiamy Państwu ofertę sprzedaży 4 - pok...
607,781,781,Przestronne 4 pok. mieszkanie w Centrum Bytomia,Bytom,18 listopada 2023,"104,38 m²",3889.63 zł/m²,406 000 zł,https://www.otodom.pl/pl/oferta/przestronne-4-...,3/3,do zamieszkania,3,Przedstawiam Państwu wyjątkową ofertę sprzedaż...


Jak możemy zauważyć zostały pobrane oferty nie tylko dla Katowic, ale także dla innych miejscowości obok Katowic. Wynika to z faktu iż na ostatniej stronie po skończeniu się ofert dla Katowic są wyświetlane również oferty z okolic. Nalezy więc usunąć te oferty.

In [6]:
df = df[df['Lokalizacja'].str.contains('Katowice')]

In [8]:
liczba_wierszy = df.shape[0]
print("Liczba wierszy:", liczba_wierszy)

Liczba wierszy: 603


In [9]:
df = df.drop(df.columns[0], axis=1)

### I. Usunięcie duplikatów

In [8]:
df.drop_duplicates(subset='Nazwa oferty', keep='first', inplace=True)
liczba_wierszy1 = df.shape[0]
print("Liczba wierszy po usunięciu duplikatów:", liczba_wierszy1)

Liczba wierszy po usunięciu duplikatów: 426


### II. Sprawdzenie braków danych i ich usunięcie 

In [11]:
braki_danych = df.isnull().sum()

print("Liczba brakujących danych w poszczególnych kolumnach:")
braki_danych

Liczba brakujących danych w poszczególnych kolumnach:


Unnamed: 0            0
Nazwa oferty          0
Lokalizacja           0
Data dodania          0
Powierzchnia w m²     0
Cena za 1m²           0
Kwota                 0
Link                  0
Poziom                6
Umeblowane           49
Liczba pokoi          0
Opis                  0
dtype: int64

In [12]:
df = df.dropna()

liczba_wierszy = df.shape[0]
print("Liczba wierszy po usunięci braków danych:", liczba_wierszy)

DataFrame po usunięciu wierszy z brakującymi danymi:


,Unnamed: 0,Nazwa oferty,Lokalizacja,Data dodania,Powierzchnia w m²,Cena za 1m²,Kwota,Link,Poziom,Umeblowane,Liczba pokoi,Opis
0,0,Mieszkanie do sprzedaży Katowice Bogucice 2 pok.,"Katowice, Bogucice",Dzisiaj o 14:16,"42,70 m²",8173.3 zł/m²,349 000 zł,https://www.otodom.pl/pl/oferta/mieszkanie-do-...,1/10,do zamieszkania,2,Oferujemy do sprzedaży 2 pokojowe mieszkanie o...
1,1,Okazja! Super mieszkanie ul. Adama! 62 m2 5 pi...,"Katowice, Giszowiec",Dzisiaj o 17:24,62 m²,5951.61 zł/m²,369 000 zł,https://www.olx.pl/d/oferta/okazja-super-miesz...,5,Nie,3 pokoje,"Agencje proszę nie dzwonić, chyba ze z zainter..."
2,2,SUPERJEDNOSTKA - zachodnie mieszkanie do wejścia,Katowice,12 grudnia 2023,"47,40 m²",9978.9 zł/m²,473 000 zł,https://www.otodom.pl/pl/oferta/superjednostka...,7/15,do zamieszkania,2,Mamy przyjemność przedstawić Państwu mieszkani...
3,3,Przestronne 3 pok.mieszkanie przy Parku Kościu...,Katowice,07 grudnia 2023,"73,38 m²",5444.26 zł/m²,399 500 zł,https://www.otodom.pl/pl/oferta/przestronne-3-...,2/4,do remontu,3,MOŻLIWOŚĆ ZAKUPU MIESZKANIA W RAMACH RZĄDOWEGO...
4,5,"Znakomite 3 pok. mieszkanie 53,4m2 Katowice-Mu...","Katowice, Murcki",25 listopada 2023,"53,37 m²",8993.82 zł/m²,480 000 zł,https://www.otodom.pl/pl/oferta/znakomite-3-po...,2/3,do zamieszkania,3,"Znakomite 3 pok. mieszkanie 53,4m2 Katowice-Mu..."


In [14]:
df.reset_index(drop=True, inplace=True)


### III. Ujednolicenie wartości w kolumn dla ofert OLX oraz OTODOM

Forma zapisu wartości dla poszczegółnych atrybutów na OLX różni się od formy zapisu na platformie OTODOM. Należy więc ujednoliczić zapis dla poprawnego przetwarzania tych danych. 

##### Kolumna 'Umeblowanie'

Wartości dla kolumny 'Umeblowanie' w ofertach OLX są zapisywane jako 'Tak' lub 'Nie', natomiast OTODOM używa zapisów takich jak: 'do remontu','do wykończenia', 'do zamieszkania'. Zinterpretowano więc że, oferta 'do zamieszkania' oznacza, że jest umeblowane, a 'do wykończenia' lub 'do remontu', że mieszkanie nie jest umeblowane. W związku z tym ujednolicono zapis na formę stosowaną przez OLX.

In [15]:
df["Umeblowane"] = df["Umeblowane"].replace({
    'do zamieszkania': 'Tak',
    'do wykończenia': 'Nie',
    'do remontu': 'Nie',
    
})

df['Umeblowane']

0      Tak
1      Nie
2      Tak
3      Nie
4      Tak
      ... 
369    Nie
370    Nie
371    Nie
372    Nie
373    Tak
Name: Umeblowane, Length: 374, dtype: object

##### Kolumna 'Liczba pokoi'

Również w przypadku kolumny 'Liczba pokoi' zapis na stronach się różni. Na OLX jest zapis jedynie  formie wartości liczbowej, natomiast na OTODOM jest dodane słowo 'pokój/pokoje'. W związku z tym zostanie wybrana jedynie wartośc liczbowa.

In [16]:
df['Liczba pokoi'] = df['Liczba pokoi'].str[0]

##### Kolumna 'Poziom'

W przypadku kolumny 'Poziom' dla ofert OTODOM poza poznaczeniem pietra na którym się znajduje mieszkanie, jest również infomacja ile jest łącznie pięter. Usunięto więc infromacje ile jest pięter, czyli wybrano tylko to co jest prze znakiem "/".

In [17]:
df["Poziom"] = df["Poziom"].str.split("/").str[0]

##### Kolumna 'Kwota'

W kolumnie 'Kwota' dla ofert OTODOM, w niektórych przypadkach jest również informacja czy cena jest do negocjacji. Aby zapis był jednolity dla obu platform usunięto tą informację.

In [18]:

df['Kwota'] = df['Kwota'].str.extract('(\d+\s?\d*)', expand=False)
df['Kwota'] 


0      349 000
1      369 000
2      473 000
3      399 500
4      480 000
        ...   
369    983 925
370    449 000
371    399 000
372    503 800
373    490 630
Name: Kwota, Length: 374, dtype: object

### IV. Usunięcie znaków tekstowych przy wartościach liczbowych w kolumnach 

W celu dalszej analizy i mozliwych porblemów z konwerowaniem danych, w kolumnach z wartościami liczbowymi takimi jak 'Cena za m2' czy 'Powierzchnia' usunięto dane tekstowe.

##### Kolumna 'Cena za m2'

In [19]:
df['Cena za 1m²'] = df['Cena za 1m²'].str.replace(' zł/m²', '')
df['Cena za 1m²']

0       8173.3
1      5951.61
2       9978.9
3      5444.26
4      8993.82
        ...   
369    8803.12
370     7080.9
371    6927.08
372    8780.06
373    8538.64
Name: Cena za 1m², Length: 374, dtype: object

##### Kolumna 'Powierzchnia w m2'

In [20]:
df['Powierzchnia w m²'] = df['Powierzchnia w m²'].str.replace(' m²', '')
df['Powierzchnia w m²']

0       42,70
1          62
2       47,40
3       73,38
4       53,37
        ...  
369    111,77
370     63,41
371     57,60
372     57,38
373     57,46
Name: Powierzchnia w m², Length: 374, dtype: object

### V. Wyodrębnienie Dzielnic w Katowicach

Po dokonaniu analizy można zauważyć, że wiele ofert nie ma jasno sprecyzowanej okolicy w której znajduje się wystawione na sprzedaż mieszkanie (istanieje jedynie nazwa'Katowice').Natomiast wiele ofert posiada  szczegółowe opisy, które już zawierają taką informacje. Zdecydowano się więc na przeprowadzenie analizy tekstu w celu wyodrębnienia nazw tych dzielnic w celu uzupełniania braków danych

#### 1. Wyodrębnienie dzielnic z tabeli 'Lokalizacja'

W pierwszym etapie z kolumny 'Lokalizacja' wyodrębniono nazwy dzilenic i usunięto kolumnę 'Lokalizacja', która nie będzie już potrzebna w dalszej części analizy

In [21]:
def process_location(location):
    if isinstance(location, str):
        parts = location.split(', ')
        if len(parts) > 1:
            return parts[1]
    return None

df['Dzielnica'] = df['Lokalizacja'].apply(process_location)

##### 2. Utworzenie słownika

 Utworzono słownik wyrazów na podstawie którego algorytm będzie przeszukiwał opis oferty w celu wyodrębnienia dzielnicy. W słowniku zawarto orginalne uniklane nazwy odzilenic oraz zazwyczaj odmieniane nazwy tych dzielnic w opisach.

In [22]:
unikalne_lokalizacje = df['Dzielnica'].unique().tolist()

In [23]:
słowa_klucz=['ligota-panewniki','podlesie','kostuchna','srodmiescie','murcki','dab','piotrowice-ochojec','bogucice','welnowiec-jozefowiec','osiedle tysiaclecia','szopienice-burowiec',
             'koszutka','dabrówka mała','osiedle paderewskiego-muchowiec','zaleze','janow-nikiszowiec','giszowiec','zawodzie','osiedle witosa','zaleska halda-brynow cz. zach.',
             'brynow-cz. wsch.-osiedle zgrzebioka','zarzecze','ligota','podlesiu','kostuchnie','srodmiesciu','murckach','dabie','piotrowicach','bogucicach','welnowcu','tysiaclecia',
             'szopienicach','koszutce','dabrowce','paderewskiego','zalezu','nikiszowcu','giszowcu','zawodziu','witosa','zaleskiej','brynowie','zarzeczu','ligota','brynow','tysiaclecie','janow','janosie'
             'nikoszowiec','welnowiec','jozefowiec','muchowiec','muchowcu','zagrzebionka','zagrzebionce']

In [24]:
dane=df['Opis']
dane

0      Oferujemy do sprzedaży 2 pokojowe mieszkanie o...
1      Agencje proszę nie dzwonić, chyba ze z zainter...
2      Mamy przyjemność przedstawić Państwu mieszkani...
3      MOŻLIWOŚĆ ZAKUPU MIESZKANIA W RAMACH RZĄDOWEGO...
4      Znakomite 3 pok. mieszkanie 53,4m2 Katowice-Mu...
                             ...                        
369    Oferuję Państwu do sprzedaży przestronne i roz...
370    Przedmiotem ogłoszenia jest 2-pokojowe mieszka...
371    Przedmiotem ogłoszenia jest 3-pokojowe mieszka...
372    1-pokojowe mieszkanie numer a.1.1.02  na 1. pi...
373    Na sprzedaż w pełni wyposażona, komfortowa kaw...
Name: Opis, Length: 374, dtype: object

##### 3. Zamiana tekstu na małe litery

In [25]:
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().splitlines()

custom_stop_words = ['f','m']

stop_words += custom_stop_words
    

In [26]:
def clean_text(text):
    text = re.sub("\s{2,}", " ", text)
    text = re.sub("(\r\n|\r|\n)", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.lower()
    text = re.sub("[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", " ", text)
    text = re.sub("\d", "", text)
    text = re.sub("\s{2,}", " ", text)
    text = re.sub(":\w+:", "", text)  # Remove emoticons in the format ":emoticon_name:"
    text = re.sub('[^\w\s]', '', text)
    return text

In [27]:
df['Opis oczyszczony'] = dane.apply(clean_text)

#### 4. Tokenizacja

In [28]:
def tokenize_text(text):
    return word_tokenize(text)

df['Opis oczyszczony'] = df['Opis oczyszczony'].apply(tokenize_text)

Przeprowadzono tokenizację tekstu w celu wyodrębnienia pojedynczych słów.

#### 5. Stopwords

In [29]:
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_words]

df['Opis oczyszczony'] = df['Opis oczyszczony'].apply(remove_stopwords)

In [30]:
oferty_5_pokoi = df[df['Liczba pokoi'] == '5']
oferty_5_pokoi

,Unnamed: 0,Nazwa oferty,Lokalizacja,Data dodania,Powierzchnia w m²,Cena za 1m²,Kwota,Link,Poziom,Umeblowane,Liczba pokoi,Opis,Dzielnica,Opis oczyszczony
25,33,Dwupoziomowe Mieszkanie Katowice,"Katowice, Piotrowice-Ochojec",12 grudnia 2023,139,5755.4,800 000,https://www.otodom.pl/pl/oferta/dwupoziomowe-m...,7,Tak,5,Szukasz mieszkania bez nakładów finansowych? ...,Piotrowice-Ochojec,"[szukasz, mieszkania, nakładów, finansowych, z..."
57,92,Apartamentowiec-dobra cena/Recepcja/Klima/CENTRUM,Katowice,12 grudnia 2023,"115,70",6913.57,799 900,https://www.otodom.pl/pl/oferta/apartamentowie...,7,Tak,5,Dzień dobry Sprzedam bardzo ładne mieszkanie w...,None,"[dzień, dobry, sprzedam, ładne, mieszkanie, ap..."
121,209,Dwupoziomowy Penthouse Katowice Ochojec,Katowice,12 grudnia 2023,"104,90",7168.73,752 000,https://www.otodom.pl/pl/oferta/dwupoziomowy-p...,7,Tak,5,Penthouse Katowice – ul. Ziołowa 43A Cena 7520...,None,"[penthouse, katowice, ul, ziołowa, cena, miesz..."
145,249,REZERWACJA. 5 kawalerek ROI 10%,"Katowice, Śródmieście",19 listopada 2023,"118,18",7268.57,859 000,https://www.otodom.pl/pl/oferta/rezerwacja-5-k...,3,Tak,5,Oferujemy pakiet 5 WYNAJĘTYCH kawalerek na spr...,Śródmieście,"[oferujemy, pakiet, wynajętych, kawalerek, spr..."
149,254,Mieszkanie - Katowice Piotrowice,"Katowice, Piotrowice-Ochojec",17 listopada 2023,"131,59",5729.92,754 000,https://www.otodom.pl/pl/oferta/mieszkanie-kat...,1,Nie,5,Oferuję do sprzedaży bezczynszowy lokal położo...,Piotrowice-Ochojec,"[oferuję, sprzedaży, bezczynszowy, lokal, poło..."
171,291,5-pokojowe mieszkanie 161m2 Bezpośrednio,"Katowice, Koszutka",17 listopada 2023,"161,24",11813.28,1 904,https://www.otodom.pl/pl/oferta/5-pokojowe-mie...,> 10,Nie,5,5-pokojowe mieszkanie numer a.1.30.03 na 30. ...,Koszutka,"[pokojowe, mieszkanie, numer, piętrze, budynku..."
214,356,5-pokojowe mieszkanie 113m2 + balkon Bez Prowizji,"Katowice, Piotrowice-Ochojec",01 grudnia 2023,"113,10",10200,1 153,https://www.otodom.pl/pl/oferta/5-pokojowe-mie...,7,Nie,5,5-pokojowe mieszkanie numer G7-02 na 7. piętr...,Piotrowice-Ochojec,"[pokojowe, mieszkanie, numer, g, piętrze, budy..."


##### 6. Przeprowadzenie wyodrębnienia słów kluczowych

In [31]:
def znajdz_slowo_klucz(opis_tokenized):
    for slowo in słowa_klucz:
        if slowo in opis_tokenized:
            return slowo
    return None

df['Słowo_klucz'] = df['Opis oczyszczony'].apply(znajdz_slowo_klucz)
df.head()

In [33]:
słowa_klucz1=['centrum','śródmieście','srodmiescie']

oferta=df['Nazwa oferty']

df['Nazwa oferty1'] = oferta.apply(clean_text)

In [34]:
def znajdz_slowo_klucz(opis_tokenized):
    for slowo in słowa_klucz1:
        if slowo in opis_tokenized:
            return slowo
    return None

df['Słowo_klucz1'] = df['Nazwa oferty1'].apply(znajdz_slowo_klucz)

##### 6. Zastąpienie braków danych w kolumnie 'Dzielnica'

Brak danych w kolumnie 'Dzielnica' zoastły zastąpione słowami kluczowym, a więc brakującymi nazwami dzielnic oraz zmieniono wyrazy aby zaczynały się z dużej litery, jak pozostały nazwy w kolumnie.

In [35]:
df['Dzielnica'] = df.apply(lambda row: row['Słowo_klucz'] if pd.isna(row['Dzielnica']) else row['Dzielnica'], axis=1).str.title()
df['Dzielnica'] = df.apply(lambda row: row['Słowo_klucz1'] if pd.isna(row['Dzielnica']) else row['Dzielnica'], axis=1).str.title()

In [37]:
dzielnice = df['Dzielnica'].unique().tolist()
dzielnice

['Bogucice',
 'Giszowiec',
 None,
 'Murcki',
 'Koszutka',
 'Janów-Nikiszowiec',
 'Kostuchna',
 'Ligota-Panewniki',
 'Osiedle Tysiąclecia',
 'Śródmieście',
 'Ligota',
 'Piotrowice-Ochojec',
 'Osiedle Witosa',
 'Brynów-Cz. Wsch.-Osiedle Zgrzebioka',
 'Paderewskiego',
 'Szopienice-Burowiec',
 'Wełnowiec-Józefowiec',
 'Załęże',
 'Centrum',
 'Brynowie',
 'Dąb',
 'Zawodzie',
 'Dąbrówka Mała',
 'Muchowiec',
 'Bogucicach',
 'Podlesie',
 'Osiedle Paderewskiego-Muchowiec']

Usunięcie zbędnych kolumn


In [38]:
df = df.drop('Słowo_klucz', axis=1)
df = df.drop('Słowo_klucz1', axis=1)
df = df.drop('Nazwa oferty1', axis=1)

Z uwagi na to, iż uzupełnione braki sa skróconymi nazwami dzielnic, zmieniono je na pełne nazwy. Dodatkowo algorytm nie odnalazł wszystkich brakujących nazw dzielnic, prawdopodbnie na unikalne przeształcenia nazw, które są używane przez ogłaszjących. Z tego względu, aby unikąc możliwych problemów poczas dlaszej analizy, usunięto także braki danych. A  Brynów zaklasyfikowano jako 1.

In [39]:

# Zamiana na pełne nazwy dzielnic 
df['Dzielnica'] = df['Dzielnica'].replace({
    'Ligota': 'Ligota-Panewniki',
    'Centrum': 'Śródmieście',
    'Srodmiescie': 'Śródmieście',
    'Brynow': 'Brynów',
    'Brynowie': 'Brynów',
    'Brynów-Cz. Wsch.-Osiedle Zgrzebioka':'Brynów',
    'Załęska Hałda-Brynów Cz. Zach.':'Brynów',
    'Zaleze': 'Załęże',
    'Tysiaclecia': 'Osiedle Tysiąclecia',
    'Paderewskiego': 'Osiedle Paderewskiego-Muchowiec',
    'Osiedle Paderewskiego': 'Osiedle Paderewskiego-Muchowiec',
    'Muchowiec': 'Osiedle Paderewskiego-Muchowiec',
    'Osiedle Paderewskiego': 'Osiedle Paderewskiego-Muchowiec',
    'Dab': 'Dąb',
    'Bogucicach': 'Bogucice'

})

In [40]:
location_counts = df['Liczba pokoi'].value_counts()
location_counts

Liczba pokoi
2    152
3    142
1     41
4     31
5      7
6      1
Name: count, dtype: int64

In [41]:
location_counts = df['Dzielnica'].value_counts()
location_counts

Dzielnica
Koszutka                           64
Śródmieście                        46
Piotrowice-Ochojec                 39
Ligota-Panewniki                   33
Wełnowiec-Józefowiec               22
Osiedle Tysiąclecia                19
Kostuchna                          16
Janów-Nikiszowiec                  13
Bogucice                           12
Załęże                             10
Dąb                                 9
Brynów                              7
Murcki                              7
Giszowiec                           6
Osiedle Paderewskiego-Muchowiec     6
Dąbrówka Mała                       6
Zawodzie                            5
Osiedle Witosa                      3
Szopienice-Burowiec                 2
Podlesie                            1
Name: count, dtype: int64

In [42]:
df['Poziom'] = df['Poziom'].replace({
    'parter': 'Parter',
    'Powyżej 10':'> 10'
})

In [43]:
df= df.dropna()

In [44]:
df.shape[0]

326

In [45]:
df = df.drop(df.columns[0], axis=1)

In [47]:
df.drop(16, inplace=True) # usunięcie tej wartosci 171


In [48]:
df.shape[0]

325

#### VI. Sprawdzenie i zmiana typu danych

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 325 entries, 0 to 373
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Nazwa oferty       325 non-null    object
 1   Lokalizacja        325 non-null    object
 2   Data dodania       325 non-null    object
 3   Powierzchnia w m²  325 non-null    object
 4   Cena za 1m²        325 non-null    object
 5   Kwota              325 non-null    object
 6   Link               325 non-null    object
 7   Poziom             325 non-null    object
 8   Umeblowane         325 non-null    object
 9   Liczba pokoi       325 non-null    object
 10  Opis               325 non-null    object
 11  Dzielnica          325 non-null    object
 12  Opis oczyszczony   325 non-null    object
dtypes: object(13)
memory usage: 35.5+ KB


In [50]:
df['Powierzchnia w m²'] = df['Powierzchnia w m²'].str.replace(',', '.')
df['Powierzchnia w m²'] = pd.to_numeric(df['Powierzchnia w m²'], errors='coerce')

df['Kwota'] = df['Kwota'].str.replace(' ', '').str.replace(',', '.')
df['Kwota'] = pd.to_numeric(df['Kwota'], errors='coerce')

df['Cena za 1m²'] = pd.to_numeric(df['Cena za 1m²'], errors='coerce')

df['Liczba pokoi'] = pd.to_numeric(df['Liczba pokoi'], errors='coerce')

In [54]:

df['Data dodania'] = df['Data dodania'].apply(lambda x: dateparser.parse(x, languages=['pl']) if pd.notna(x) else pd.NaT)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 325 entries, 0 to 373
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Nazwa oferty       325 non-null    object        
 1   Lokalizacja        325 non-null    object        
 2   Data dodania       325 non-null    datetime64[ns]
 3   Powierzchnia w m²  325 non-null    float64       
 4   Cena za 1m²        325 non-null    float64       
 5   Kwota              325 non-null    int64         
 6   Link               325 non-null    object        
 7   Poziom             325 non-null    object        
 8   Umeblowane         325 non-null    object        
 9   Liczba pokoi       325 non-null    int64         
 10  Opis               325 non-null    object        
 11  Dzielnica          325 non-null    object        
 12  Opis oczyszczony   325 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(8)
memory usage: 35

#### VII. Zapisanie danych do CSV

In [57]:
df.to_csv('Cleaned_df_17g_stare_nowe_cleaned3.csv')